In [51]:
import sqlite3
import sys
# from IAST import IAST
# import pandas as pd


In [50]:
class IAST():
    # iast to inidc language 
    # keyword which are need to tokenize input <iast>
    vowel_plist=[['r̥̄', 'l̥̄'], 
                 ['r̥', 'au', 'ai', 'ụ̄ ', 'ạ̄ ', 'oṁ', 'm̐', 'aḥ', 'l̥'], 
                 ['a', 'ā', 'ạ', 'ụ', 'æ', 'ǣ', 'i', 'ī', 'u', 'ū', 'e', 'ē', 'ê', 'ê',
                  'o', 'ǒ', 'ō', 'ô', 'ʻ', 'ḥ', 'ḫ', 'ẖ', 'ṁ', 'ṃ']]
    
    consonant_list = [['n̆g', 'n̆j', 'n̆ḍ', 'n̆d', 'm̆b', 'k͟h'], 
                       ['kh', 'g̈', 'gh', 'ch', 'ĉh', 'jh', 'ṭh', 'ḍh', 'dh', 'd̤', 
                        'ṛh', 'th', 'ph', 'bh', 'b̤', 'ṟ̄', 'y̌', 'r̆', 'l̤', '||'], 
                       ['ḵ', 'k', 'g', 'ṅ', 'c', 'ĉ', 'j', 'ǰ', 'ĵ', 'ñ', 'ṭ', 'ḍ', 'ḍ', 
                        'ṛ', 'ṇ', 't', 'd', 'n', 'p', 'b', 'm', 'ṟ', 'ṯ', 'ḏ', 'ṉ',
                        'ḻ', 'y', 'ẏ', 'r', 'l', 'ḷ', 'v', 'ś', 'ṣ', 's', 'h', 'q', 'ġ', 
                        'z', 'z', 'ž', 'ž', 'ž', 'f', 's̱', 's̤', 'h̤', 't̤', 'w',
                        'ẕ', 'ż', 'ẓ', 'ẏ', 'ṟ', 
                        '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '|']]
    
    # Phonetic Search Static Variables
    zero_vowels={ '':['a', "ā", "â","i", "ī","u", "ū",chr(805),
                      "l̥", "l̥̄","e", "ē", "ê","o", "ō", "ô",
                      "ṁ", "m̐", "ṃ", "ṃ","n̆", "n̆", "n̆","ḥ" , "ḫ", "ẖ", "ḥ"],
                  'r': ["r̥", "r̥̄"]
                } # replacing with r is not working for 'r̥' so we replace with chr(805) above
    truncated_vowels = {
                        'a':["ā", "â"], 
                        'i':["i", "ī"], 
                        'u':["u", "ū"], 
                        'r':["r̥", "r̥̄"],
                        'l':["l̥", "l̥̄"],
                        "e":["e", "ē", "ê"],
                                # "ai", 
                        "o": ["o", "ō", "ô"], 
                                                # "au",
                        'm' :["ṁ", "m̐", "ṃ", "ṃ"], 
                        'n': ["n̆", "n̆", "n̆"], 
                        'h' :["ḥ" , "ḫ", "ẖ", "ḥ"],
                        }
    basic_truncated_consonat = {
                        'k' : ['ḵ', 'k', 'kh','k͟ha'],
                        'g' :['g','g̈','gh','ġ'],
                        'n' : ['ṅ','n̆','ñ','ṇ','ṉ'],
                        'c' : ['c', 'ĉ','ch','ĉh'],
                        'j' : ['j','ǰ', 'ĵ', 'jh'],
                        't': ['ṭ','ṭa','t','th','ṯ','t̤'],
                        'd' : ['ḍ', 'd̤','ḍ','ḍh','d','dh','ḏ'],
                        'p' : ['p', 'ph'],
                        'b' : [ 'b', 'b̤', 'bh'],
                        'm' : ['m̆' ],
                        'r' : ['ṟ', 'r̆'],
                        'l' :['ḻ', 'ḷ', 'l̤'],
                        'y' : ['y', 'ẏ', 'y̌'],
                        's': ['ś', 'ṣ', 's','s̱', 's̤','sh' ],
                        "z": ["z","ž","ž","ž",'ẕ','ẕ','ẓ','ż'],
                        'h' : ['h','h̤']
                        }
    
    
    def __init__(self,db_path,table_name_alpha,table_name_barakadi ):
        self.db_path = db_path
        self.alphabet = table_name_alpha
        self.barakhadi = table_name_barakadi
        self.db_connect = sqlite3.connect(db_path)
        self.halant_list = self.get_halant_list() #  ['्', '্', '્', '್', '്', '୍', '్']
# phonetic search algo inicialization
    def set_query(self,query):
        db_cursor = self.db_connect.cursor()
        try :
            db_cursor.execute(query)    
        except sqlite3.Error as e:
            print('SQLite error: %s' % (' '.join(e.args)))
            print("Exception class is: ", e.__class__)
            print('SQLite traceback: ')
            exc_type, exc_value, exc_tb = sys.exc_info()
            # data = []
        
        finally:
            self.db_connect.commit()
            # auto_increment and delete on cascade https://stackoverflow.com/questions/29037793/sqlite-integrityerror-unique-constraint-failed
            sefl.db_cursor.close()
            # db_cursor.close()
            # print('Read Query!')
        # return data
    
    def get_query(self,query): # all query which will give or get the data or query which will return some value
        db_cursor = self.db_connect.cursor()
        try :
            db_cursor.execute(query)    
            columns = [column[0] for column in db_cursor.description]
            data = [dict(zip(columns, row)) for row in db_cursor.fetchall()]
            # self.db_connect.close()
        except sqlite3.Error as e:
            print('SQLite error: %s' % (' '.join(e.args)))
            print("Exception class is: ", e.__class__)
            print('SQLite traceback: ')
            exc_type, exc_value, exc_tb = sys.exc_info()
            data = []
        
        finally:
            # db_connect.commit()
            # auto_increment and delete on cascade https://stackoverflow.com/questions/29037793/sqlite-integrityerror-unique-constraint-failed
            db_cursor.close()
            # db_cursor.close()
            # print('Read Query!')
        return data
    
    def get_iast_idx_query(letter,talbe_name): # query: given indic letter will return iast mapped value 
        query =f"""
        SELECT * FROM {talbe_name} 
        WHERE Devanagari == '{letter}'
            OR Bengali–Assamese == '{letter}'
            OR Gujarati == '{letter}'
            OR Kannada == '{letter}'
            OR Malayalam == '{letter}'
            OR Odia == '{letter}'
            OR Tamil == '{letter}'
            OR Telugu == '{letter}';
        """
        # print(query.replace('\n','').replace("  ","").replace('OR',' OR')) 
    # SELECT * FROM IndianAlphabet WHERE Devanagari == 'ध' OR Bengali–Assamese == 'ध' OR Gujarati == 'ध' OR Kannada == 'ध' OR Malayalam == 'ध' OR Odia == 'ध' OR Tamil == 'ध' OR Telugu == 'ध'    
        return query 

    def get_halant_list (self):
        letter = 'ŭ'
        query = f"SELECT * FROM {self.barakhadi} WHERE IAST='{letter}'" 
        data =self.get_query(query)
        del data[0]['IAST']
        halant_list = []
        for value in data[0].values():
            if value is not None and value not in halant_list:
                halant_list.append(value)
        # print(halant_list)
        return halant_list

    # word = 'ధృత్రాష్ట్ర ఉవాచ'
    def convert2iast(self,word): # arg can be word, sentance, line, para, whole doc
        output_token = ''
        for letter in word: # word
            query = IAST.get_iast_idx_query(letter,self.alphabet)    
            alpha_token =self.get_query(query)
            
            query = IAST.get_iast_idx_query(letter,self.barakhadi)    
            barakhadi_token =self.get_query(query)
            
            if len(alpha_token) !=0:
                output_token += alpha_token[0]['IAST']
                # print(alpha_token[0]['IAST'],end='')
            elif letter in  self.halant_list and output_token[-1] in "a": # if 'क '
                output_token = output_token[:-1]
            elif len(barakhadi_token) !=0 and output_token[-1] in "a":
                output_token = output_token[:-1]+ barakhadi_token[0]['IAST']
            elif len(barakhadi_token) !=0 :
                output_token += barakhadi_token[0]['IAST'].replace("a",'')
            elif len(barakhadi_token) ==0  and len(alpha_token) ==0 and ord(letter)==8205: # cleaing data
                pass
            else:
                # print(f"""NOT Present in alpha and barakadi{letter}=={ord(letter)} """)
                output_token += letter
            # print(out)
            # print(f'{letter}\t| {output_token}')
        # print(f'{word}\t| {output_token}')
        return output_token
    # iast.convert2iast( word)

    def debug_letterbyletter(self,text):
    # text =' ധർമക്ഷേത്രേ കുരുക്ഷേത്രേ സമവേതാ യുയുത്സവഃ ।' #	| dhaർmakṣētrē kurukṣētrē samavētā yuyutsavaḥ |  
        for letter in text: 
            query = IAST.get_iast_idx_query(letter,self.alphabet)    
            alpha_token =self.get_query(query)
            
            query = IAST.get_iast_idx_query(letter,self.barakhadi)    
            barakhadi_token =self.get_query(query)
            
            output_token=''
            if len(alpha_token) !=0:
                output_token += alpha_token[0]['IAST']
                # print(alpha_token[0]['IAST'],end='')
            elif len(barakhadi_token) !=0 :
                output_token = output_token[:-1]+ barakhadi_token[0]['IAST']
            else:
                output_token = letter
                
            output = f'letter= {letter} \t| ascii(letter) ={ord(letter[0])}\t| iast = {output_token}' #  | ascii(iast)={ord(iast_letter)}'
            print(output)

# Methods for phonetic search
    # Replace many to one
    
    def replace_m2o(text, source=None, dest=None): # 
        if isinstance(source, list):
            for source_letter in source:
                text = text.replace(source_letter,dest)
        elif isinstance(source, str):
            text = text.replace(source,dest)
        # print(text)
        return text
        
    # Replace many to many
    def replace_m2m(output_data,info_dict):
        for dest in info_dict.keys():
            source = info_dict[dest]
            output_data = IAST.replace_m2o(output_data, source=source, dest=dest)
        return output_data

    # Basic Stemming
    def basic_hash(iast_text): # if text is in hin,kan,tel,mal,guj,..etc need to convert to iast 
        basic_stem_dict = iast.zero_vowels
        basic_stem_dict.update(iast.basic_truncated_consonat)
        output =IAST.replace_m2m(iast_text,basic_stem_dict)
        return output
        
    # Normal Stemming
    def normal_hash(iast_text):
        normal_stem_dict = iast.truncated_vowels
        normal_stem_dict.update(iast.basic_truncated_consonat)
        output = IAST.replace_m2m(iast_text,normal_stem_dict)
        return output
    
    def get_indic_symbol_query(iast_letter,language,table_name): # query given indic letter will return iast mapped value 
        # query =f"""
        # SELECT type {language} FROM {table_name} 
        # WHERE IAST LIKE '{iast_letter}%';
        # """
        query =f"""SELECT type, IAST, {language}  FROM {table_name} 
        WHERE IAST LIKE '{iast_letter}%';
        """
 # 'type': 'consonants',
 #  'IAST': 'gha',
 #  'Devanagari': 'घ',
 #  'Bengali–Assamese': 'ঘ',
 #  'Gujarati': 'ઘ',
 #  'Gurmukhi': 'ਘ',
        # print(query.replace('\n','').replace("  ","")) 
    # SELECT * FROM IndianAlphabet WHERE Devanagari == 'ध' OR Bengali–Assamese == 'ध' OR Gujarati == 'ध' OR Kannada == 'ध' OR Malayalam == 'ध' OR Odia == 'ध' OR Tamil == 'ध' OR Telugu == 'ध'    
        return query 
        
# iast to indic language(any lang) 
    
    def lex_iast(keyword, word):
        tokens=[]
        # print(keyword,word)
        slic_pstart = 0 # previous start point
        # slic_pstart = 0 # previous start point
        # slic/_pstart = 0 # previous start point
        
        len_word = len(word)
        slic3_flag = False
        slic2_flag = False
        slic1_flag = False
        
        for idx, letter in enumerate(word):
            slic3 = word[idx:idx+3]
            if slic3 in keyword[0]:
                slic3_flag=True
            else:
                slic3_flag=False
            slic2 = word[idx:idx+2]            
            if slic2 in keyword[1]:
                slic2_flag=True
            else:
                slic2_flag=False        
            slic1 = word[idx:idx+1]        
            if slic1 in keyword[2]:
                slic1_flag=True
            else:
                slic1_flag=False
        
            if slic3_flag:
                if slic_pstart < idx:
                    # print(f'Append missing data btw idx slic3 {slic_pstart}:{idx} {word[slic_pstart:idx]}' )                                                    
                    tokens.append(word[slic_pstart:idx])
                tokens.append(slic3)                
                slic_pstart=idx+1 + len(slic3)-1
                # print(f'At index {idx} :Need to split3 at {slic3}' )
            else:
                if slic2_flag:
                    if slic_pstart < idx:
                        # print(f'Append missing data btw idx slic2 {slic_pstart}:{idx} {word[slic_pstart:idx]}' )                                    
                        tokens.append(word[slic_pstart:idx])            
                    # if slic_pstart <idx+1:
                    tokens.append(slic2)
                    slic_pstart=idx+len(slic2)       
                    # print(f'slic2 pstart:{slic_pstart}')
                    # print(f'At index {idx} :Need to split2 at {slic2} and set next start point: {idx+1+len(slic2)} and it value:{word[idx+len(slic2)]}' )                
                else:
                    if slic1_flag:
                        if slic_pstart < idx:
                            tokens.append(word[slic_pstart:idx])
                            # print(f'Append missing data btw idx slic1 {slic_pstart}:{idx} {word[slic_pstart:idx]}' )                                    
                        if slic_pstart<=idx:
                            tokens.append(slic1)
                            slic_pstart=idx+1 + len(slic1)-1
                            # print(f'At index {idx} :Need to split1 at {slic2}' )                                    
        return tokens

    def iast2tokens(vowel_plist,consonant_list,  word):
        iast_tokens= []
        vowel_tokens = IAST.lex_iast(vowel_plist,word)
        # print(vowel_tokens)
        if word[-1*len(vowel_tokens[-1]):]==vowel_tokens[-1]:
            pass
            # print('Last word match with vowel no need to append')
        else:
            vowel_tokens.append(word.split(vowel_tokens[-1])[-1])
            # print('need to append')
            # print(vowel_tokens)
        for i in vowel_tokens:
            # print(i, lex_iast(consonant_list,i))
            if len(IAST.lex_iast(consonant_list,i)) <=1:
                iast_tokens.append(i)
            else:
                iast_tokens.extend(IAST.lex_iast(consonant_list,i))
        return iast_tokens


    def tokens2dict_tokenes(self,tokens,indic_lang):
        input_tokens=''
        output_string = []
        for token in tokens:
            query_bara = f"""SELECT IAST,{indic_lang} FROM {self.barakhadi} WHERE IAST LIKE '%{token}'"""
            query_alpha = f"""SELECT type, IAST,{indic_lang} FROM {self.alphabet} WHERE IAST LIKE '{token}%'"""
            # print(query_bara)
            # print(query_alpha)
            data_alpha = iast.get_query(query_alpha)
            data_bara = iast.get_query(query_bara)
            # print(data_bara)
            input_tokens += token+ ' '
            # print(token,'Alphabets: ', data_alpha,' Barakadi',data_bara)
            temp_dic = dict()
            temp_dic['IAST']=token
            temp_dic['lang']=indic_lang
            
            # temp_dic['type']=
            # temp_dic['alph']=
            # temp_dic['bara']= 
            if len(data_alpha):
                temp_dic['type']=data_alpha[0]['type']
                # temp_dic['alph']=data_alpha[0][indic_lang] # wrong method if token = n ,n̆ḍa, n̆ja then : 
                # we 1st search result is none which we need to filter
                for entry in data_alpha:
                    if token =='r':
                        # print(entry)
                        if entry['IAST']=='ra':
                            temp_dic['alph']=entry[indic_lang]
                            temp_dic['type']=entry['type']
                            # print(temp_dic)                    
                            break        

                    if token =='l':
                        if entry['IAST']=='la':
                            temp_dic['alph']=entry[indic_lang]
                            temp_dic['type']=entry['type']
                            # print(temp_dic)                    
                            break        
                        
                    if entry[indic_lang] is not None and( token!='r') and ( token!='l') :
                        # print(entry['Telugu'],entry['IAST'],entry['type'])
                        temp_dic['alph']=entry[indic_lang]
                        temp_dic['type']=entry['type']
                        break        
                # output_string +=' | '+ data_alpha[0][indic_lang]+' : '  +data_alpha[0]['type'] +' | '
            else:
                temp_dic['alph']=None
            if len(data_bara):
                # output_string +=' | '+ data_bara[0][indic_lang] +' | '
                temp_dic['type']='vowel'
                temp_dic['bara']=data_bara[0][indic_lang]
            else:
                temp_dic['bara']=None
            output_string.append(temp_dic)
        return output_string
        
    def get_indic_halant(self,indic_lang):
        query_alpha = f"""SELECT IAST,{indic_lang} FROM {self.barakhadi} WHERE IAST='ŭ'"""
        data_alpha = iast.get_query(query_alpha)
        halant = iast.get_query(query_alpha)[0][indic_lang]
        return halant

    
    def dict_tokens2indic(output_string,halant):
        output=''
        for idx, item in enumerate(output_string):
            print_status = False    
            # print(idx, item)
            if idx ==0:
                prev_item=dict()
            else:
                prev_item=output_string[idx-1]
            if idx < len(output_string)-1:
                
                next_item = output_string[idx+1]
            elif idx ==len(output_string)-1:
                next_item = dict()
                
            if 'type' in item.keys() and item['type']=='consonants':
                if 'type' in next_item.keys() and next_item['type']=='vowel':
                    # print(item['alph'], end=" ")
                    output +=item['alph']
                    print_status =True
                elif 'type' in next_item.keys() and next_item['type']=='consonants':
                    # print(item['alph']+halant,end="")
                    output +=item['alph']+halant
                    print_status =True
                elif 'type' not in next_item: # word ending with consonant and halant
                    output +=item['alph']+halant
                    print_status =True
                    
                    
            if 'type' in item.keys() and item['type']=='vowel':
                # print('ITEM: ',item)
                # print('PREV ITEM: ',prev_item)
                if 'type' in prev_item.keys() and prev_item['type']=='consonants':
                    # print(item['bara'], end=' ')
                    if item['IAST']=='a':
                        print_status =True                            
                        pass
                    else:        
                        output +=item['bara']
                        print_status =True            
                    # print(item)

                if 'type' in prev_item.keys() and prev_item['type']=='vowel':
                    output +=item['bara']            
                    # print(item)
                    print_status =True
                # pass
                if 'type' not in prev_item : # starting of word or starting of line
                    if item['alph'] is not None :
                        output +=item['alph']                            
                        print_status =True                
        
            
            if not print_status:
                output +=item['IAST']
            # print(output)
        return output

    def iast2indic(self,vowel_plist,consonant_list,word,indic_lang):
        if len(word)==0:
            return word
        tokens= IAST.iast2tokens(vowel_plist,consonant_list,  word)
        # print(tokens)
        dict_tokene_list = self.tokens2dict_tokenes(tokens,indic_lang)
        # print(output_string)
        halant=self.get_indic_halant(indic_lang)
        # print(halant)
        output=IAST.dict_tokens2indic(dict_tokene_list,halant)
        return output



In [40]:
# db_path = 'iast-token.db'    
db_path ='iast-map-modified.db'

In [41]:
# db_path = 'iast-token.db'    
# db_path = 'iast-map-modified.db'
table_name_alpha= 'IndianAlphabet'
table_name_barakadi= 'Barakhadi'
iast = IAST(db_path,table_name_alpha,table_name_barakadi)

In [36]:
print(iast.alphabet) # IndianAlphabet
print(iast.barakhadi)  # Barakhadi
query = IAST.get_iast_idx_query('ध',iast.alphabet)
print(query.replace('\n','').replace("  ","").replace('OR',' OR')) 
# SELECT * FROM IndianAlphabet 
    # WHERE Devanagari == 'ध' 
        # OR Bengali–Assamese == 'ध' OR Gujarati == 'ध' 
        # OR Kannada == 'ध' OR Malayalam == 'ध' 
        # OR Odia == 'ध' OR Tamil == 'ध' 
        # OR Telugu == 'ध';
print(iast.get_query(query))
# [{'IAST': 'dha', 'Devanagari': 'ध', 'Bengali–Assamese': 'ধ', 'Gujarati': 'ધ', 'Gurmukhi': 'ਧ', 'Kannada': 'ಧ', 'Malayalam': 'ധ', 'Nastaliq': 'دھ', 'Odia': 'ଧ', 'Sinhala': 'ධ', 'Tamil': None, 'Telugu': 'ధ'}]


IndianAlphabet
Barakhadi
SELECT * FROM IndianAlphabet WHERE Devanagari == 'ध' OR Bengali–Assamese == 'ध' OR Gujarati == 'ध' OR Kannada == 'ध' OR Malayalam == 'ध' OR Odia == 'ध' OR Tamil == 'ध' OR Telugu == 'ध';
[{'type': 'consonants', 'IAST': 'dha', 'Devanagari': 'ध', 'Bengali–Assamese': 'ধ', 'Gujarati': 'ધ', 'Gurmukhi': 'ਧ', 'Kannada': 'ಧ', 'Malayalam': 'ധ', 'Nastaliq': 'دھ', 'Odia': 'ଧ', 'Sinhala': 'ධ', 'Tamil': None, 'Telugu': 'ధ'}]


In [6]:
print(iast.get_halant_list()) # ['्', '্', '્', '್', '്', '୍', '్']
print(iast.convert2iast('ధృత్రాష్ట్ర ఉవాచ'))
# iast.halant_list

['्', '্', '્', '್', '്', '୍', '్']
dhr̥trāṣṭra uvāca


In [7]:
iast.get_query(query)

[{'type': 'consonants',
  'IAST': 'dha',
  'Devanagari': 'ध',
  'Bengali–Assamese': 'ধ',
  'Gujarati': 'ધ',
  'Gurmukhi': 'ਧ',
  'Kannada': 'ಧ',
  'Malayalam': 'ധ',
  'Nastaliq': 'دھ',
  'Odia': 'ଧ',
  'Sinhala': 'ධ',
  'Tamil': None,
  'Telugu': 'ధ'}]

In [8]:
# text = '''ଧୃତରାଷ୍ଟ୍ର ଉଵାଚ |\nধৃতরাষ্ট্র উবাচ |\nધૃતરાષ્ટ્ર ઉવાચ |\nத்றுதராஷ்ட்ர உவாச |'''
# text = 'TAM= ta்rutarasa்ta்ra' # uvaca|'
text = 'TEL= dr̥trstr vc |'
print(iast.convert2iast(text))
iast.debug_letterbyletter(text)


TEL= dr̥trstr vc |
letter= T 	| ascii(letter) =84	| iast = T
letter= E 	| ascii(letter) =69	| iast = E
letter= L 	| ascii(letter) =76	| iast = L
letter= = 	| ascii(letter) =61	| iast = =
letter=   	| ascii(letter) =32	| iast =  
letter= d 	| ascii(letter) =100	| iast = d
letter= r 	| ascii(letter) =114	| iast = r
letter= ̥ 	| ascii(letter) =805	| iast = ̥
letter= t 	| ascii(letter) =116	| iast = t
letter= r 	| ascii(letter) =114	| iast = r
letter= s 	| ascii(letter) =115	| iast = s
letter= t 	| ascii(letter) =116	| iast = t
letter= r 	| ascii(letter) =114	| iast = r
letter=   	| ascii(letter) =32	| iast =  
letter= v 	| ascii(letter) =118	| iast = v
letter= c 	| ascii(letter) =99	| iast = c
letter=   	| ascii(letter) =32	| iast =  
letter= | 	| ascii(letter) =124	| iast = |


In [10]:
# for line in text_telugu.strip().split('\n'):
# source : https://hinduliterature.org/tamil/scriptures/srimad_bhagawad_gita_chapter_1.php
file_name ='bg-hinid'
with open (file_name,'r') as f:
   f_hindi = f.read()  

file_name ='bg-kannada'
with open (file_name,'r') as f:
   f_kanada = f.read()  

file_name ='bg-malayalam'
with open (file_name,'r') as f:
   f_malayalam = f.read()  

file_name ='bg-telugu'
with open (file_name,'r') as f:
   f_telugu = f.read()  

file_name ='bg-oriya'
with open (file_name,'r') as f:
   f_oriya = f.read()  

file_name ='bg-bengali'
with open (file_name,'r') as f:
   f_bengali = f.read()  

file_name ='bg-gujarati'
with open (file_name,'r') as f:
   f_gujarati = f.read()  

file_name ='bg-tamil'
with open (file_name,'r') as f:
   f_tamil = f.read()  

print(len( f_telugu.split('\n')),len(f_kanada.split('\n')), len(f_malayalam.split('\n')), len(f_hindi.split('\n')  ))

min_len  = min(len( f_telugu.split('\n')),len(f_kanada.split('\n')), len(f_malayalam.split('\n')), len(f_hindi.split('\n')  ))
print(f"""LANGUAGE\t| Origian Language Script \t| IAST CONVERT Script """)
for line_idx in range(min_len-1):
    print(line_idx)
    print("-"*40)
    line = f_telugu.strip().split('\n')[line_idx]
    if len(line):
        print(f"""TEL\t| {line.strip()}\t| {iast.convert2iast( line)} """)
    line = f_kanada.strip().split('\n')[line_idx]
    if len(line):
        print(f"""KAN\t| {line.strip()}\t| {iast.convert2iast( line)} """)
    line = f_malayalam.strip().split('\n')[line_idx]
    if len(line):
        print(f"""MAL\t| {line.strip()}\t| {iast.convert2iast( line)} """)
    line = f_hindi.strip().split('\n')[line_idx]
    if len(line):
        print(f"""HIN\t| {line.strip()}\t| {iast.convert2iast( line)} """)
        # print(f""" {iast.convert2iast( line)} """)
    line = f_oriya.strip().split('\n')[line_idx]
    if len(line):
        print(f"""ORI\t| {line.strip()}\t| {iast.convert2iast( line)} """)

    line = f_bengali.strip().split('\n')[line_idx]
    if len(line):
        print(f"""BEN\t| {line.strip()}\t| {iast.convert2iast( line)} """)
        
    line = f_gujarati.strip().split('\n')[line_idx]
    if len(line):
        print(f"""GUJ\t| {line.strip()}\t| {iast.convert2iast( line)} """)

    line = f_tamil.strip().split('\n')[line_idx]
    if len(line):
        print(f"""TAM\t| {line.strip()}\t| {iast.convert2iast( line)} """)

157 469 157 140
LANGUAGE	| Origian Language Script 	| IAST CONVERT Script 
0
----------------------------------------
TEL	| ధృతరాష్ట్ర ఉవాచ |	| dhr̥tarāṣṭra uvāca | 
KAN	| ಧೃತರಾಷ್ಟ್ರ ಉವಾಚ ।	| dhr̥tarāṣṭra uvāca | 
MAL	| ധൃതരാഷ്ട്ര ഉവാച ।	| dhr̥tarāṣṭra uvāca | 
HIN	| धृतराष्ट्र उवाच	| dhr̥tarāṣṭra uvāca 
ORI	| ଧୃତରାଷ୍ଟ୍ର ଉଵାଚ |	| dhr̥tarāṣṭra uvāca | 
BEN	| ধৃতরাষ্ট্র উবাচ |	| dhr̥tarāṣṭra ubāca | 
GUJ	| ધૃતરાષ્ટ્ર ઉવાચ |	| dhr̥tarāṣṭra uvāca | 
TAM	| த்றுதராஷ்ட்ர உவாச |	| ta்ṟutarāṣa்ṭa்ra uvāca | 
1
----------------------------------------
KAN	| ಧರ್ಮಕ್ಷೇತ್ರೇ ಕುರುಕ್ಷೇತ್ರೇ ಸಮವೇತಾ ಯುಯುತ್ಸವಃ ।	| dharmakṣētrē kurukṣētrē samavētā yuyutsavaḥ | 
MAL	| ധർമക്ഷേത്രേ കുരുക്ഷേത്രേ സമവേതാ യുയുത്സവഃ ।	| dhaṟmakṣētrē kurukṣētrē samavētā yuyutsavaḥ | 
HIN	| धर्मक्षेत्रे कुरुक्षेत्रे समवेता युयुत्सव: ।	| dharmakṣētrē kurukṣētrē samavētā yuyutsavaḥ | 
ORI	| ଧର୍ମକ୍ଷେତ୍ରେ କୁରୁକ୍ଷେତ୍ରେ ସମଵେତା ୟୁୟୁତ୍ସଵଃ |	| dharmakṣētrē kurukṣētrē samavētā ẏuẏutsavaḥ | 
BEN	| ধর্মক্ষেত্রে কুরুক্ষেত্রে সমবেতা য়ুয়ুত্সবঃ |	| 

In [11]:
text = """
రామేశ్వర
ராமேஸ்வரா
ರಾಮೇಶ್ವರ
രാമേശ്വര
रामेश्वरा
रामेश्‍वर
ରମେଶ୍ୱରା
রামেশ্বরা
રામેશ્વર

Guṇasandhi (Vowel raising)
రామ + ఈశ్వరం = రామేశ్వరం
ರಾಮ + ಈಶ್ವರ  = ರಾಮೇಶ್ವರಂ
ராம + ஈஸ்வர = ராமேஸ்வரம்
രാമ + ഈശ്വര = രാമേശ്വരം
राम + ईश्वर = रामेश्वरम
ରାମା + ଇଶ୍ୱରା = ରମେଶ୍ୱରମ୍ 
রাম + এশ্বর = রামেশ্বরম
રામ + ઈશ્વરા = રામેશ્વરમ
राम + ईश्वर = रामेश्वरम्
राम शब्द रूप https://www.youtube.com/watch?v=oRjXNGIG7-4
विभक्ति	एकवचन	द्विवचन	बहुवचन
प्रथमा	रामः	रामौ	रामाः
द्वितीया	रामम्	रामौ	रामान्
तृतीया	रामेण	रामाभ्याम्	रामैः
चतुर्थी	रामाय	रामाभ्याम्	रामेभ्यः
पंचमी	रामात्	रामाभ्याम्	रामेभ्यः
षष्‍ठी	रामस्य	रामयोः	रामाणाम्
सप्‍तमी	रामे	रामयोः	रामेषु
सम्बोधन	हे राम!	हे रामौ!	हे रामाः!

राम अच्छा बालक है
రామ్ మంచి అబ్బాయి
ರಾಮ್ ಒಳ್ಳೆಯ ಹುಡುಗ
റാം നല്ല കുട്ടിയാണ്
ରାମ ଜଣେ ଭଲ ବାଳକ
રામ સારો છોકરો છે
রাম একটি ভালো ছেলে

सीता अच्छी लड़की है
సీత అచ్ఛీ అమ్మాయి

"""

f_tamil = """ 

ആലായാല്‍ തറ വേണം  അടുത്തൊരമ്പലം വേണം
ആലിനു ചേര്‍ന്നൊരു കുളവും വേണം
കുളിപ്പാനായ് കുളം വേണം  കുളത്തില്‍ ചെന്താമര വേണം
കുളിച്ച് ചെന്നകം പുക്കാന്‍ ചന്ദനം വേണം

പൂവായാല്‍ മണം വേണം  പൂമാനായാല്‍ ഗുണം വേണം
പൂമാനിനിമാര്‍കള്‍ അടക്കം വേണം

യുദ്ധത്തിങ്കല്‍ രാമന്‍ നല്ലൂ, കുലത്തിങ്കല്‍ സീത നല്ലൂ
ഊണുറക്കമുപേക്ഷിക്കാന്‍ ലക്ഷ്മണന്‍ നല്ലൂ
പടയ്ക്ക് ഭരതന്‍ നല്ലൂ, പറവാന്‍ പൈങ്കിളി നല്ലൂ
പറക്കുന്ന പക്ഷികളില്‍ ഗരുഢന്‍ നല്ലൂ

നാടായാല്‍ നൃപന്‍ വേണം  അരികില്‍ മന്ത്രിമാര്‍ വേണം
നാടിനു ഗൂണമുള്ള പ്രജകള്‍ വേണം..

മങ്ങാട്ടച്ചനു ന്യായം നല്ലൂ മംഗല്യത്തിനു സ്വര്‍ണ്ണേ നല്ലൂ
മങ്ങാതിരിപ്പാന്‍ നിലവിളക്ക് നല്ലൂ.

പാല്യത്തച്ചനുപായം നല്ലൂ പാലില്‍ പഞ്ചസാര നല്ലൂ
പാരാതിരിപ്പാന്‍ ചില പദവി നല്ലൂ
"""
# f_tamil = text
lines = f_tamil.strip().split('\n')
for line in lines:
    if len(line):
        print(f"""{iast.convert2iast( line)} """)
        # print(f"""MAL \t {line.strip()}\t {iast.convert2iast( line)} """)

ālāyāl taṟa vēṇaṁ  aṭuttorampalaṁ vēṇaṁ 
ālinu cērnnoru kuḷavuṁ vēṇaṁ 
kuḷippānāy kuḷaṁ vēṇaṁ  kuḷattil centāmara vēṇaṁ 
kuḷicc cennakaṁ pukkān candanaṁ vēṇaṁ 
pūvāyāl maṇaṁ vēṇaṁ  pūmānāyāl guṇaṁ vēṇaṁ 
pūmāninimārkaḷ aṭakkaṁ vēṇaṁ 
yuddhattiṅkal rāman nallū, kulattiṅkal sīta nallū 
ūṇuṟakkamupēkṣikkān lakṣmaṇan nallū 
paṭaykk bharatan nallū, paṟavān paiṅkiḷi nallū 
paṟakkunna pakṣikaḷil garuḍhan nallū 
nāṭāyāl nr̥pan vēṇaṁ  arikil mantrimār vēṇaṁ 
nāṭinu gūṇamuḷḷa prajakaḷ vēṇaṁ.. 
maṅṅāṭṭaccanu nyāyaṁ nallū maṁgalyattinu svarṇṇē nallū 
maṅṅātirippān nilaviḷakk nallū. 
pālyattaccanupāyaṁ nallū pālil pañcasāra nallū 
pārātirippān cila padavi nallū 


In [11]:
search_word = ' Dhritirashtra ' # dhr̥tarāṣṭra uvāca
search_word = 'kṛṣṇa'
print("# Original Text:", search_word)
search_word = search_word.strip().lower()
print('# # # text converted to iast format')
# convert2iast
search_iast = iast.convert2iast(search_word) # similar to idempotent matrx no loss of info if ':' not present
print(search_iast)

print('# # # BASIC HASHING')
print(IAST.basic_hash(search_iast))

print('# # # NORMAL HASHING')
print(IAST.normal_hash(search_iast))

db_data = """
TEL= ధృతరాష్ట్ర ఉవాచ |
KAN= ಧೃತರಾಷ್ಟ್ರ ಉವಾಚ ।	 
MAL= ധൃതരാഷ്ട്ര ഉവാച ।	 
HIN= धृतराष्ट्र उवाच	|
ORI= ଧୃତରାଷ୍ଟ୍ର ଉଵାଚ | 
BEN= ধৃতরাষ্ট্র উবাচ |	 
GUJ= ધૃતરાષ્ટ્ર ઉવાચ |	
TAM= த்றுதராஷ்ட்ர உவாச |	
"""

dest = 's'
source = iast.basic_truncated_consonat[dest]
# print(f'source= {source},\ndest= {dest}')

data_iast = iast.convert2iast(db_data)
print('# # # text converted to iast format')
print(data_iast)

# steam_token =IAST.replace_m2o(data_token, source=source, dest='s')
# print(steam_token)


# # Basic Stemming
# basic_stem_dict = iast.zero_vowels
# basic_stem_dict.update(iast.basic_truncated_consonat)
# print(IAST.replace_m2m(data_token,basic_stem_dict))
print('# # # BASIC STEM TEXT')
print(IAST.basic_stem_text(data_iast))
# 
# normal_stem_dict = iast.truncated_vowels
# normal_stem_dict.update(iast.basic_truncated_consonat)
# print(IAST.replace_m2m(data_token,normal_stem_dict))
print('# # # NORMAL STEM TEXT')
print(IAST.normal_hash(data_iast))

# Original Text: kṛṣṇa
# # # text converted to iast format
kṛṣṇa
# # # BASIC HASHING
kṛṣṇ
# # # NORMAL HASHING
kṛṣṇa
# # # text converted to iast format

TEL= dhr̥tarāṣṭra uvāca |
KAN= dhr̥tarāṣṭra uvāca |	 
MAL= dhr̥tarāṣṭra uvāca |	 
HIN= dhr̥tarāṣṭra uvāca	|
ORI= dhr̥tarāṣṭra uvāca | 
BEN= dhr̥tarāṣṭra ubāca |	 
GUJ= dhr̥tarāṣṭra uvāca |	
TAM= ta்ṟutarāṣa்ṭa்ra uvāca |	

# # # BASIC STEM TEXT


AttributeError: type object 'IAST' has no attribute 'basic_stem_text'

In [12]:
# Original Text:  Dhritirashtra 
# # # text converted to iast format
dhritirashtra
# # # BASIC STEM TEXT
drtrstr
# # # NORMAL STEM TEXT
dritirastra
# # # text converted to iast format


NameError: name 'dhritirashtra' is not defined

In [13]:
# import pandas as pd 
# # # # # # # # #
# # Export: db to excel
# # # # # # # # #  
# # pd.read_sql_table(iast.alphabet, iast.db_connect)
# alphabets= pd.read_sql_query(f"SELECT * FROM {iast.alphabet}", iast.db_connect)
# barakhadi = pd.read_sql_query(f"SELECT * FROM {iast.barakhadi}", iast.db_connect)

# with pd.ExcelWriter('IASTv2.xlsx', mode="w",engine="openpyxl")as writer:
#     alphabets.to_excel(writer, sheet_name=iast.alphabet,index=False)
#     barakhadi.to_excel(writer,sheet_name=iast.barakhadi,index=False)

In [1]:
# Import db for excel 
import pandas as pd
import sqlite3

path = 'IAST-modified.xlsx'

# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
alphabets = pd.read_excel(path,sheet_name='IndianAlphabet')
# alphabets.drop(['index'], axis=1, inplace=True) 
# alphabets.set_index('ISO',inplace=True)
# print(alphabets.columns) # 'index '  
# display(alphabets)
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
barakhadi = pd.read_excel(path,sheet_name='Barakhadi')
# print(barakhadi.columns)
# display(barakhadi)
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 

connect = sqlite3.connect('iast-map-modified.db')
# alphabets.to_sql('IndianAlphabet', connect, if_exists='replace',index=False)
# barakhadi.to_sql('Barakhadi', connect, if_exists='replace',index=False)

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [45]:
# def get_indic_symbol_query(iast_letter,language,table_name): # query given indic letter will return iast mapped value 
#     query =f"""
#     SELECT {language} FROM {table_name} 
#     WHERE IAST == '{iast_letter}';
#     """
#     print(query.replace('\n','').replace("  ","").replace('OR',' OR')) 
# # SELECT * FROM IndianAlphabet WHERE Devanagari == 'ध' OR Bengali–Assamese == 'ध' OR Gujarati == 'ध' OR Kannada == 'ध' OR Malayalam == 'ध' OR Odia == 'ध' OR Tamil == 'ध' OR Telugu == 'ध'    
#     return query 
print(IAST.get_indic_symbol_query('ka','telugu',iast.alphabet).replace('\n','').replace("  ",""))

SELECT type, IAST, teluguFROM IndianAlphabet WHERE IAST LIKE 'ka%';
SELECT type, IAST, teluguFROM IndianAlphabet WHERE IAST LIKE 'ka%';


In [51]:
query = IAST.get_indic_symbol_query('gh','Kannada',iast.alphabet) # SELECT telugu FROM IndianAlphabet WHERE IAST == 'k';
iast.get_query(query)[0]['Kannada']

SELECT type, IAST, KannadaFROM IndianAlphabet WHERE IAST LIKE 'gh%';


'ಘ'

In [16]:
# word = 'dhr̥tarāṣṭra uvāca'

# letter  = word[]
print('क्षत्रिय')

क्षत्रिय


In [17]:
word = 'क्षत्रिय'
# # word = 'oṁ'
# word = """ṛ  ṝ  ṛa  ṛha  śṛṇu ṛ ṝ "r̥" "r̥̄" ḷ  ḹ "l̥", "l̥̄" 'a':["ā", "â"], 
#                                     'i':["i", "ī"], 
#                                     'u':["u", "ū"], 
#                                     'r':["r̥", "r̥̄"],
#                                     'l':["l̥", "l̥̄"],
#                                     "e":["e", "ē", "ê"],
#                                             # "ai", 
#                                     "o": ["o", "ō", "ô"], 
#                                                             # "au",
#                                     'm' :["ṁ", "m̐", "ṃ", "ṃ"], 
#                                     'n': ["n̆", "n̆", "n̆"], 
#                                     'h' :["ḥ" , "ḫ", "ẖ", "ḥ"], """.replace(' ','').replace('"','').replace("'",'').replace(',','').replace(':','')
# for i in word :
#     print(i,ord(i), end='\n')

In [18]:
# word[0] + chr(2381) + 'अ'
# 'क' + '्' + 'आ' # => 'क्' + 'आ' = 'का' 


In [19]:
iast.convert2iast(word)

'kṣatriya'

In [20]:
data = iast.get_query("SELECT type, IAST FROM IndianAlphabet ")
df = pd.DataFrame(data)

for i in list(df['IAST']):
    print(i, len(i))

NameError: name 'pd' is not defined

In [12]:
vowel_plist = []
# 3:
temp = ['r̥̄' ,'l̥̄' ]  
vowel_plist.append(temp)
# 2 
temp = ['r̥' ,'au', 'ai', 'ụ̄ ', 'ạ̄ ', 'oṁ','m̐','aḥ','l̥']
vowel_plist.append(temp)

 # 'ḷ', 'ḹ',
temp = ['a' ,'ā' ,'ạ', 'ụ', 'æ', 'ǣ', 'i' ,'ī', 'u', 'ū', 'e', 'ē', 'ê', 'ê', 'o', 'ǒ', 'ō', 'ô', 'ʻ','ḥ','ḫ','ẖ','ṁ','ṃ' ]

vowel_plist.append(temp)

print(vowel_plist) 

[['r̥̄', 'l̥̄'], ['r̥', 'au', 'ai', 'ụ̄ ', 'ạ̄ ', 'oṁ', 'm̐', 'aḥ', 'l̥'], ['a', 'ā', 'ạ', 'ụ', 'æ', 'ǣ', 'i', 'ī', 'u', 'ū', 'e', 'ē', 'ê', 'ê', 'o', 'ǒ', 'ō', 'ô', 'ʻ', 'ḥ', 'ḫ', 'ẖ', 'ṁ', 'ṃ']]


In [48]:
vowel_plist=[['r̥̄', 'l̥̄'], 
 ['r̥', 'au', 'ai', 'ụ̄ ', 'ạ̄ ', 'oṁ', 'm̐', 'aḥ', 'l̥'], 
 ['a', 'ā', 'ạ', 'ụ', 'æ', 'ǣ', 'i', 'ī', 'u', 'ū', 'e', 'ē', 'ê', 'ê', 'o', 'ǒ', 'ō', 'ô', 'ʻ', 'ḥ', 'ḫ', 'ẖ', 'ṁ', 'ṃ']]
temp

[['r̥̄', 'l̥̄'],
 ['r̥', 'au', 'ai', 'ụ̄ ', 'ạ̄ ', 'oṁ', 'm̐', 'aḥ', 'l̥'],
 ['a',
  'ā',
  'ạ',
  'ụ',
  'æ',
  'ǣ',
  'i',
  'ī',
  'u',
  'ū',
  'e',
  'ē',
  'ê',
  'ê',
  'o',
  'ǒ',
  'ō',
  'ô',
  'ʻ',
  'ḥ',
  'ḫ',
  'ẖ',
  'ṁ',
  'ṃ']]

In [84]:
# ord(vowel_plist[1][-3])
# # ord('ṃ')
# 'm̐' =='m̐'
# # # 'ṃ'=='ṃ'
# for i in ['ḥ','ḫ','ẖ','ṃ','ḥ']:
# print(len('ê'))
# 'ê'=='ê'

In [13]:
consonant_list = []
#  3
temp = ['n̆g' ,'n̆j', 'n̆ḍ' ,'n̆d', 'm̆b' ,'k͟h' ]
consonant_list.append(temp)
# 2
temp = ['kh' , 'g̈' , 'gh' , 'ch' , 'ĉh' , 'jh' , 'ṭh' ,'ḍh',  'dh' , 'd̤', 'ṛh' , 'th' , 'ph' ,   'bh' ,'b̤',  'ṟ̄',  'y̌' , 'r̆',  'l̤','||' ]
consonant_list.append(temp)
# 1
temp = ['ḵ', 'k', 'g', 'ṅ', 'c', 'ĉ', 'j', 'ǰ', 'ĵ', 'ñ', 'ṭ', 'ḍ', 'ḍ' ,'ṛ', 'ṇ',
't', 'd', 'n', 'p', 'b', 'm', 'ṟ', 'ṯ', 'ḏ' ,'ṉ', 'ḻ', 'y', 'ẏ',  'r',
'l' ,'ḷ', 'v', 'ś', 'ṣ' ,'s', 'h' ,'q', 'ġ' ,'z', 'z', 'ž', 'ž', 'ž', 'f' ,
's̱', 's̤', 'h̤' ,'t̤', 'w' ,'ẕ','ż' ,'ẓ', 'ẏ' ,'ṟ', '0', '1' ,'2' ,'3', '4', '5', '6', '7', '8', '9','|'  ]
consonant_list.append(temp)
print(consonant_list, len(consonant_list))

[['n̆g', 'n̆j', 'n̆ḍ', 'n̆d', 'm̆b', 'k͟h'], ['kh', 'g̈', 'gh', 'ch', 'ĉh', 'jh', 'ṭh', 'ḍh', 'dh', 'd̤', 'ṛh', 'th', 'ph', 'bh', 'b̤', 'ṟ̄', 'y̌', 'r̆', 'l̤', '||'], ['ḵ', 'k', 'g', 'ṅ', 'c', 'ĉ', 'j', 'ǰ', 'ĵ', 'ñ', 'ṭ', 'ḍ', 'ḍ', 'ṛ', 'ṇ', 't', 'd', 'n', 'p', 'b', 'm', 'ṟ', 'ṯ', 'ḏ', 'ṉ', 'ḻ', 'y', 'ẏ', 'r', 'l', 'ḷ', 'v', 'ś', 'ṣ', 's', 'h', 'q', 'ġ', 'z', 'z', 'ž', 'ž', 'ž', 'f', 's̱', 's̤', 'h̤', 't̤', 'w', 'ẕ', 'ż', 'ẓ', 'ẏ', 'ṟ', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '|']] 3


In [49]:
consonant_list = [['n̆g', 'n̆j', 'n̆ḍ', 'n̆d', 'm̆b', 'k͟h'], 
                   ['kh', 'g̈', 'gh', 'ch', 'ĉh', 'jh', 'ṭh', 'ḍh', 'dh', 'd̤', 
                    'ṛh', 'th', 'ph', 'bh', 'b̤', 'ṟ̄', 'y̌', 'r̆', 'l̤', '||'], 
                   ['ḵ', 'k', 'g', 'ṅ', 'c', 'ĉ', 'j', 'ǰ', 'ĵ', 'ñ', 'ṭ', 'ḍ', 'ḍ', 
                    'ṛ', 'ṇ', 't', 'd', 'n', 'p', 'b', 'm', 'ṟ', 'ṯ', 'ḏ', 'ṉ',
                    'ḻ', 'y', 'ẏ', 'r', 'l', 'ḷ', 'v', 'ś', 'ṣ', 's', 'h', 'q', 'ġ', 
                    'z', 'z', 'ž', 'ž', 'ž', 'f', 's̱', 's̤', 'h̤', 't̤', 'w',
                    'ẕ', 'ż', 'ẓ', 'ẏ', 'ṟ', 
                    '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '|']]
# consonant_listp

[['n̆g', 'n̆j', 'n̆ḍ', 'n̆d', 'm̆b', 'k͟h'],
 ['kh',
  'g̈',
  'gh',
  'ch',
  'ĉh',
  'jh',
  'ṭh',
  'ḍh',
  'dh',
  'd̤',
  'ṛh',
  'th',
  'ph',
  'bh',
  'b̤',
  'ṟ̄',
  'y̌',
  'r̆',
  'l̤',
  '||'],
 ['ḵ',
  'k',
  'g',
  'ṅ',
  'c',
  'ĉ',
  'j',
  'ǰ',
  'ĵ',
  'ñ',
  'ṭ',
  'ḍ',
  'ḍ',
  'ṛ',
  'ṇ',
  't',
  'd',
  'n',
  'p',
  'b',
  'm',
  'ṟ',
  'ṯ',
  'ḏ',
  'ṉ',
  'ḻ',
  'y',
  'ẏ',
  'r',
  'l',
  'ḷ',
  'v',
  'ś',
  'ṣ',
  's',
  'h',
  'q',
  'ġ',
  'z',
  'z',
  'ž',
  'ž',
  'ž',
  'f',
  's̱',
  's̤',
  'h̤',
  't̤',
  'w',
  'ẕ',
  'ż',
  'ẓ',
  'ẏ',
  'ṟ',
  '0',
  '1',
  '2',
  '3',
  '4',
  '5',
  '6',
  '7',
  '8',
  '9',
  '|']]

In [14]:
word = 'kaṁ  itāḥ kiṁ  yuyutsavaḥ kl̥̄ kl̥ pāṇḍavānīkaṁ itāḥ kiṁ āṁ  īṁ  kiṁ yuyutsuṁ rānsakhīṁstathā'
# ord('ṁ')
print(vowel_plist)
print(IAST.iast2tokens(vowel_plist, consonant_list, word) )

[['r̥̄', 'l̥̄'], ['r̥', 'au', 'ai', 'ụ̄ ', 'ạ̄ ', 'oṁ', 'm̐', 'aḥ', 'l̥'], ['a', 'ā', 'ạ', 'ụ', 'æ', 'ǣ', 'i', 'ī', 'u', 'ū', 'e', 'ē', 'ê', 'ê', 'o', 'ǒ', 'ō', 'ô', 'ʻ', 'ḥ', 'ḫ', 'ẖ', 'ṁ', 'ṃ']]
['k', 'a', 'ṁ', '  ', 'i', 't', 'ā', 'ḥ', ' ', 'k', 'i', 'ṁ', '  ', 'y', 'u', 'y', 'u', 't', 's', 'a', 'v', 'aḥ', ' ', 'k', 'l̥̄', ' ', 'k', 'l̥', ' ', 'p', 'ā', 'ṇ', 'ḍ', 'a', 'v', 'ā', 'n', 'ī', 'k', 'a', 'ṁ', ' ', 'i', 't', 'ā', 'ḥ', ' ', 'k', 'i', 'ṁ', ' ', 'ā', 'ṁ', '  ', 'ī', 'ṁ', '  ', 'k', 'i', 'ṁ', ' ', 'y', 'u', 'y', 'u', 't', 's', 'u', 'ṁ', ' ', 'r', 'ā', 'n', 's', 'a', 'kh', 'ī', 'ṁ', 's', 't', 'a', 'th', 'ā']


In [16]:
# len('aṁ')
# print(IAST.lex_iast(consonant_list,word) )
# print(IAST.lex_iast(vowel_plist,word) )
# print( )
# SELECT type, IAST, Telugu  FROM IndianAlphabet WHERE IAST LIKE 'i%';
tokens= IAST.iast2tokens(vowel_plist,consonant_list,  word)




indic_lang = 'Telugu' # 'Kannada' # 'Telugu'
tokens = IAST.iast2tokens(vowel_plist, consonant_list, word) 
# print(tokens)
output_string = iast.tokens2dict_tokenes(tokens,indic_lang)
# print(output_string)
for i in output_string:
    print(i)
# output_string

{'IAST': 'k', 'lang': 'Telugu', 'type': 'consonants', 'alph': 'క', 'bara': None}
{'IAST': 'a', 'lang': 'Telugu', 'type': 'vowel', 'alph': 'అ', 'bara': None}
{'IAST': 'ṁ', 'lang': 'Telugu', 'alph': None, 'type': 'vowel', 'bara': 'ం'}
{'IAST': '  ', 'lang': 'Telugu', 'alph': None, 'bara': None}
{'IAST': 'i', 'lang': 'Telugu', 'type': 'vowel', 'alph': 'ఇ', 'bara': 'ి'}
{'IAST': 't', 'lang': 'Telugu', 'type': 'consonants', 'alph': 'త', 'bara': None}
{'IAST': 'ā', 'lang': 'Telugu', 'type': 'vowel', 'alph': 'ఆ', 'bara': 'ా'}
{'IAST': 'ḥ', 'lang': 'Telugu', 'alph': None, 'type': 'vowel', 'bara': 'ః'}
{'IAST': ' ', 'lang': 'Telugu', 'alph': None, 'bara': None}
{'IAST': 'k', 'lang': 'Telugu', 'type': 'consonants', 'alph': 'క', 'bara': None}
{'IAST': 'i', 'lang': 'Telugu', 'type': 'vowel', 'alph': 'ఇ', 'bara': 'ి'}
{'IAST': 'ṁ', 'lang': 'Telugu', 'alph': None, 'type': 'vowel', 'bara': 'ం'}
{'IAST': '  ', 'lang': 'Telugu', 'alph': None, 'bara': None}
{'IAST': 'y', 'lang': 'Telugu', 'type': 'conso

In [97]:
'क'+  'ा'+'ः' #>>>  'काः'
'क'+'्'   # >>> 'क्'
'क्'+ 'ा' # >>> 'क्ा'


'काः'

In [27]:
# # input_tokens
# output_string
# # len('ḥ')
# import pandas as p

# halant/viramam
# 'क'+  'ा'+'ः' #>>>  'काः'
# 'क'+'्'   # >>> 'क्'
# 'क्'+ 'ा' # >>> 'क्ा'
# input iast, outputstring
word = 'kaṁ  itāḥ kiṁ  yuyutsavaḥ  pāṇḍavānīkaṁ itāḥ kiṁ āṁ  īṁ  kiṁ yuyutsuṁ rānsakhīṁstathā'
indic_lang = 'Telugu' # 'Kannada' # 'Telugu'
# def iast2indic(iast,vowel_plist,consonant_list,word,indic_lang):
#     tokens= IAST.iast2tokens(vowel_plist,consonant_list,  word)
#     # print(tokens)
#     dict_tokene_list = iast.tokens2dict_tokenes(tokens,indic_lang)
#     # print(output_string)
#     halant=iast.get_indic_halant(indic_lang)
#     # print(halant)
#     output=IAST.dict_tokens2indic(dict_tokene_list,halant)
#     return output
iast.iast2indic(vowel_plist,consonant_list,word,indic_lang)
# print(len('l̥'),len('l̥̄'))
# 
# len('ḹ')
# l̥
# l̥̄

'కం  ఇతాః కిం  యుయుత్సవః  పాణ్డవానీకం ఇతాః కిం ఆం  ఈం  కిం యుయుత్సుం రాన్సఖీంస్తథా'

In [47]:
text = """ālāyāl taṟa vēṇaṁ aṭuttorampalaṁ vēṇaṁ
ālinu cērnnoru kuḷavuṁ vēṇaṁ 
kuḷippānāy kuḷaṁ vēṇaṁ  kuḷattil centāmara vēṇaṁ 
kuḷicc cennakaṁ pukkān candanaṁ vēṇaṁ 
pūvāyāl maṇaṁ vēṇaṁ  pūmānāyāl guṇaṁ vēṇaṁ 
pūmāninimārkaḷ aṭakkaṁ vēṇaṁ 
yuddhattiṅkal rāman nallū, kulattiṅkal sīta nallū 
ūṇuṟakkamupēkṣikkān lakṣmaṇan nallū 
paṭaykk bharatan nallū, paṟavān paiṅkiḷi nallū 
paṟakkunna pakṣikaḷil garuḍhan nallū 
nāṭāyāl nr̥pan vēṇaṁ  arikil mantrimār vēṇaṁ 
nāṭinu gūṇamuḷḷa prajakaḷ vēṇaṁ.. 
maṅṅāṭṭaccanu nyāyaṁ nallū maṁgalyattinu svarṇṇē nallū 
maṅṅātirippān nilaviḷakk nallū. 
pālyattaccanupāyaṁ nallū pālil pañcasāra nallū 
pārātirippān cila padavi nallū 
"""
# print(text.split('\n'))
# iast.iast2indic(vowel_plist,consonant_list,text,indic_lang)
# indic_lang='Devanagari'
# indic_lang='Kannada'
# indic_lang='Bengali–Assamese'
# indic_lang='Odia'
# indic_lang='Tamil'
indic_lang='Telugu'
for line in text.split('\n'):
    print(line)
    print(line.split(" "))
    for word in line.split(" "):
        # print('\nword:',word, 'len of word:',len(word))
        output =iast.iast2indic(vowel_plist,consonant_list,word,indic_lang)
#         # print(l)
        print(output, end=' ')
    


ālāyāl taṟa vēṇaṁ aṭuttorampalaṁ vēṇaṁ
['ālāyāl', 'taṟa', 'vēṇaṁ', 'aṭuttorampalaṁ', 'vēṇaṁ']
ఆలాయాల్ తఱ వేణం అటుత్తొరమ్పలం వేణం ālinu cērnnoru kuḷavuṁ vēṇaṁ 
['ālinu', 'cērnnoru', 'kuḷavuṁ', 'vēṇaṁ', '']
ఆలిను చేర్న్నొరు కుళవుం వేణం  kuḷippānāy kuḷaṁ vēṇaṁ  kuḷattil centāmara vēṇaṁ 
['kuḷippānāy', 'kuḷaṁ', 'vēṇaṁ', '', 'kuḷattil', 'centāmara', 'vēṇaṁ', '']
కుళిప్పానాయ్ కుళం వేణం  కుళత్తిల్ చెన్తామర వేణం  kuḷicc cennakaṁ pukkān candanaṁ vēṇaṁ 
['kuḷicc', 'cennakaṁ', 'pukkān', 'candanaṁ', 'vēṇaṁ', '']
కుళిచ్చ్ చెన్నకం పుక్కాన్ చన్దనం వేణం  pūvāyāl maṇaṁ vēṇaṁ  pūmānāyāl guṇaṁ vēṇaṁ 
['pūvāyāl', 'maṇaṁ', 'vēṇaṁ', '', 'pūmānāyāl', 'guṇaṁ', 'vēṇaṁ', '']
పూవాయాల్ మణం వేణం  పూమానాయాల్ గుణం వేణం  pūmāninimārkaḷ aṭakkaṁ vēṇaṁ 
['pūmāninimārkaḷ', 'aṭakkaṁ', 'vēṇaṁ', '']
పూమానినిమార్కళ్ అటక్కం వేణం  yuddhattiṅkal rāman nallū, kulattiṅkal sīta nallū 
['yuddhattiṅkal', 'rāman', 'nallū,', 'kulattiṅkal', 'sīta', 'nallū', '']
యుద్ధత్తిఙ్కల్ రామన్ నల్లూ, కులత్తిఙ్కల్ సీత నల్లూ  ūṇuṟakkamupēkṣikkān lak

In [72]:
query_alpha = f"""SELECT IAST,{indic_lang} FROM {iast.barakhadi} WHERE IAST='ŭ'"""
# query_alpha = f"""SELECT type, IAST,{indic_lang} FROM {iast.alphabet} WHERE IAST LIKE 'n%'"""
# # print(query_bara)
# print(query_alpha)
data_alpha = iast.get_query(query_alpha)
print(data_alpha[0][indic_lang])
# for entry in data_alpha:
#     if entry['Telugu'] is not None:
#         print(entry['Telugu'],entry['IAST'],entry['type'])
#         break

# data_bara = iast.get_query(query_bara)

్


In [31]:
query_bara = f"""SELECT * FROM Barakhadi WHERE IAST LIKE 'ṁ'"""
query_alpha = f"""SELECT type, IAST,{indic_lang} FROM {iast.alphabet} WHERE IAST LIKE '{token}%'"""
# query_alpha = IAST.get_indic_symbol_query(token,indic_lang,iast.alphabet) # SELECT telugu FROM IndianAlphabet WHERE IAST == 'k';
# print(query_alpha)
# query_bara = IAST.get_indic_symbol_query(token,indic_lang,iast.barakhadi) # SELECT telugu FROM IndianAlphabet WHERE IAST == 'k';
import pandas as pd
# pd.DataFrame(
iast.get_query(query_bara)
# )

[{'IAST': 'ṁ',
  'Devanagari': 'ं',
  'Bengali–Assamese': 'ং',
  'Gujarati': 'ં',
  'Gurmukhi': 'ਂ',
  'Kannada': 'ಂ',
  'Malayalam': 'ം',
  'Nastaliq': 'ں',
  'Odia': 'ଂ',
  'Sinhala': 'ං',
  'Tamil': 'ஂ',
  'Telugu': 'ం'}]

In [19]:
'n̆'=='n̆'
# 'ఁ' ==
'ఁ'=='ఁ'

True

In [9]:
text = """ālāyāl taṟa vēṇaṁ  aṭuttorampalaṁ vēṇaṁ 
ālinu cērnnoru kuḷavuṁ vēṇaṁ 
kuḷippānāy kuḷaṁ vēṇaṁ  kuḷattil centāmara vēṇaṁ 
kuḷicc cennakaṁ pukkān candanaṁ vēṇaṁ 
pūvāyāl maṇaṁ vēṇaṁ  pūmānāyāl guṇaṁ vēṇaṁ 
pūmāninimārkaḷ aṭakkaṁ vēṇaṁ 
yuddhattiṅkal rāman nallū, kulattiṅkal sīta nallū 
ūṇuṟakkamupēkṣikkān lakṣmaṇan nallū 
paṭaykk bharatan nallū, paṟavān paiṅkiḷi nallū 
paṟakkunna pakṣikaḷil garuḍhan nallū 
nāṭāyāl nr̥pan vēṇaṁ  arikil mantrimār vēṇaṁ 
nāṭinu gūṇamuḷḷa prajakaḷ vēṇaṁ.. 
maṅṅāṭṭaccanu nyāyaṁ nallū maṁgalyattinu svarṇṇē nallū 
maṅṅātirippān nilaviḷakk nallū. 
pālyattaccanupāyaṁ nallū pālil pañcasāra nallū 
pārātirippān cila padavi nallū 
"""
text = """avīt īṁ camūm ryavān ahārathāḥ vitāḥ akṣitam akṣitam
dhr̥tarāṣṭra uvāca 
 dharmakṣētrē kurukṣētrē samavētā yuyutsavaḥ | 
 māmakāḥ pāṇḍavāścaiva kimakurvata sañjaya || 1 || 
 sañjaya uvāca 
 dr̥ṣṭvā tu pāṇḍavānīkaṁ vyūḍhaṁ duryodhanastadā | 
 ācāryamupasaṅgamya rājā vacanamabravīt || 2 || 
 paśyaitāṁ pāṇḍuputrāṇāmācārya mahatīṁ camūm | 
 vyūḍhāṁ drupadaputrēṇa tava śiṣyēṇa dhīmatā || 3 || 
 atra śūrā mahēṣvāsā bhīmārjunasamā yudhi | 
 yuyudhāno virāṭaśca drupadaśca mahārathaḥ || 4 || 
 dhr̥ṣṭakētuścēkitānaḥ kāśirājaśca vīryavān | 
 purujitkuntibhojaśca śaibyaśca narapuṅgavaḥ || 5 || 
 yudhāmanyuśca vikrānta uttamaujāśca vīryavān | 
 saubhadro draupadēyāśca sarva ēva mahārathāḥ || 6 || 
 asmākaṁ tu viśiṣṭā yē tānnibodha dvijottama | 
 nāyakā mama sainyasya saṁjñārtha tānbravīmi tē || 7 || 
 bhavānbhīṣmaśca karṇaśca kr̥paśca samitiṁjayaḥ | 
 aśvatthāmā vikarṇaśca saumadattistathaiva ca || 8 || 
 anyē ca bahavaḥ śūrā madarthē tyaktajīvitāḥ | 
 nānāśastrapraharaṇāḥ sarvē yuddhaviśāradāḥ || 9 || 
 aparyāptaṁ tadasmākaṁ balaṁ bhīṣmābhirakṣitam | 
 paryāptaṁ tvidamētēṣāṁ balaṁ bhīmābhirakṣitam || 10 || 
 ayanēṣu ca sarvēṣu yathābhāgavamasthitāḥ | 
 bhīṣmamēvābhirakṣantu bhavantaḥ sarva ēva hi || 11 || 
 tasya sañjanayanharṣaṁ kuruvr̥ddhaḥ pitāmahaḥ | 
 siṁhanādaṁ vinadyoccaiḥ śaṅkhaṁ dadhmau pratāpavān || 12 || 
 tataḥ śaṅkhāśca bhēryaśca paṇavānakagomukhāḥ | 
 sahasaivābhyahanyanta sa śabdastumuloʼbhavat || 13 || 
 tataḥ śvētairhayairyuktē mahati syandanē sthitau | 
 mādhavaḥ pāṇḍavaścaiva divyau śaṅkhau pradadhmatuḥ || 14 || 
 pāñcajanyaṁ hr̥ṣīkēśo dēvadattaṁ dhanañjayaḥ | 
 pauṇḍraṁ dadhmau mahāśaṅkhaṁ bhīmakarmā vr̥kodaraḥ || 15 || 
 anantavijayaṁ rājā kuntīputro yudhiṣṭhiraḥ | 
 nakulaḥ sahadēvaśca sughoṣamaṇipuṣpakau || 16 || 
 kāśyaśca paramēṣvāsaḥ śikhaṇḍī ca mahārathaḥ | 
 dhr̥ṣṭadyumno virāṭaśca sātyakiścāparājitaḥ || 17 || 
 drupado draupadēyāśca sarvaśaḥ pr̥thivīpatē | 
 saubhadraśca mahābāhuḥ śaṅkhāndadhmuḥ pr̥thakpr̥thak || 18 || 
 sa ghoṣo dhārtarāṣṭrāṇāṁ hr̥dayāni vyadārayat | 
 nabhaśca pr̥thivīṁ caiva tumuloʼbhyanunādayan || 19 || 
 atha vyavasthitāndr̥ṣṭvā dhārtarāṣṭrānkapidhvajaḥ | 
 pravr̥ttē śastrasampātē dhanurudyamya pāṇḍavaḥ | 
 hr̥ṣīkēśaṁ tadā vākyamidamāha mahīpatē || 20 || 
 arjuna uvāca 
 sēnayorubhayormadhyē rathaṁ sthāpaya mēʼcyuta | 
 yāvadētānnirīkṣēʼhaṁ yoddhukāmānavasthitān || 21 || 
 kairmayā saha yoddhavyamasminraṇasamudyamē || 22 || 
 yotsyamānānavēkṣēʼhaṁ ya ētēʼtra samāgatāḥ | 
 dhārtarāṣṭrasya durbuddhēryuddhē priyacikīrṣavaḥ || 23 || 
 sañjaya uvāca 
 ēvamukto hr̥ṣīkēśo guḍākēśēna bhārata | 
 sēnayorubhayormadhyē sthāpayitvā rathottamam || 24 || 
 bhīṣmadroṇapramukhataḥ sarvēṣāṁ ca mahīkṣitām | 
 uvāca pārtha paśyaitānsamavētānkuruniti || 25 || 
 tatrāpaśyatsthitānpārthaḥ pitr̥̄natha pitāmahān| 
 ācāryānmātulānbhrātr̥̄nputrānpautrānsakhīṁstathā| 
 śvaśurānsuhr̥daścaiva sēnayorubhayorapi || 26 || 
 tānsamīkṣya sa kauntēyaḥ sarvānbandhūnavasthitān | 
 kr̥payā parayāviṣṭo viṣīdannidamabravīt || 27 || 
 arjuna uvāca 
 dr̥ṣṭvēmaṁ svajanaṁ kr̥ṣṇa yuyutsuṁ samupasthitam | 
 sīdanti mama gātrāṇi mukhaṁ ca pariśuṣyati || 28 || 
 vēpathuśca śarīrē mē romaharṣaśca jāyatē | 
 gāṇḍīvaṁ sraṁsatē hastāttvakcaiva paridahyatē || 29 || 
 na ca śaknomyavasthātuṁ bhramatīva ca mē manaḥ | 
 nimittāni ca paśyāmi viparītāni kēśava || 30 || 
 na ca śrēyoʼnupaśyāmi hatvā svajanamāhavē | 
 na kāṅkṣē vijayaṁ kr̥ṣṇa na ca rājyaṁ sukhāni ca || 31 || 
 kiṁ no rājyēna govinda kiṁ bhogairjīvitēna vā | 
 yēṣāmarthē kāṅkṣitaṁ no rājyaṁ bhogāḥ sukhāni ca || 32 || 
 ta imēʼvasthitā yuddhē prāṇāṁstyaktvā dhanāni ca | 
 ācāryāḥ pitaraḥ putrāstathaiva ca pitāmahāḥ || 33 || 
 mātulāḥ śvaśurāḥ pautrāḥ śyālāḥ sambandhinastathā | 
 ētānna hantumicchāmi ghnatoʼpi madhusūdana || 34 || 
 api trailokyarājyasya hētoḥ kiṁ nu mahīkr̥tē | 
 nihatya dhārtarāṣṭrānnaḥ kā prītiḥ syājjanārdana || 35 || 
 pāpamēvāśrayēdasmānhatvaitānātatāyinaḥ | """

for i in text.split('\n'):
    print(i.strip())
    print(' '.join(IAST.iast2tokens(vowel_plist,consonant_list,  i) ) )

# 

avīt īṁ camūm ryavān ahārathāḥ vitāḥ akṣitam akṣitam
a v ī t  ī ṁ  c a m ū m   r y a v ā n  a h ā r a th ā ḥ  v i t ā ḥ  a k ṣ i t a m  a k ṣ i t a m
dhr̥tarāṣṭra uvāca
dh r̥ t a r ā ṣ ṭ r a   u v ā c a  
dharmakṣētrē kurukṣētrē samavētā yuyutsavaḥ |
  dh a r m a k ṣ ē t r ē   k u r u k ṣ ē t r ē   s a m a v ē t ā   y u y u t s a v aḥ   |
māmakāḥ pāṇḍavāścaiva kimakurvata sañjaya || 1 ||
  m ā m a k ā ḥ  p ā ṇ ḍ a v ā ś c ai v a   k i m a k u r v a t a   s a ñ j a y a   ||   1   ||
sañjaya uvāca
  s a ñ j a y a   u v ā c a  
dr̥ṣṭvā tu pāṇḍavānīkaṁ vyūḍhaṁ duryodhanastadā |
  d r̥ ṣ ṭ v ā   t u   p ā ṇ ḍ a v ā n ī k aṁ   v y ū ḍh aṁ   d u r y o dh a n a s t a d ā   |
ācāryamupasaṅgamya rājā vacanamabravīt || 2 ||
  ā c ā r y a m u p a s a ṅ g a m y a   r ā j ā   v a c a n a m a b r a v ī t   ||   2   ||
paśyaitāṁ pāṇḍuputrāṇāmācārya mahatīṁ camūm |
  p a ś y ai t ā ṁ  p ā ṇ ḍ u p u t r ā ṇ ā m ā c ā r y a   m a h a t ī ṁ  c a m ū m   |
vyūḍhāṁ drupadaputrēṇa tava śiṣyēṇa dhīmatā || 3 |